In [1]:
#Understanding the data set
import pandas as pd
import numpy as np

def inspect_x_test(file_path):
    try:
        df = pd.read_csv(file_path)
        print("Read with commas")
    except pd.errors.ParserError:
        try:
            df = pd.read_csv(file_path, delimiter='\t')
            print("Read with tabs")
        except Exception as e:
            print(f"Error reading the .txt file: {e}")
            return
    except Exception as e:
        print(f"Error reading the file: {e}")
        return

    print("Data Preview:")
    print(f"Number of rows (samples): {df.shape[0]}")
    print(f"Number of columns (features): {df.shape[1]}")
    print("\nColumn names:")
    print(df.columns.tolist())

# inspect_x_test("Data/x_test.txt")
# inspect_x_test("Data/y_test.txt")


def preview_large_csv(file_path, num_lines=20, delimiter=','):
    try:
        df = pd.read_csv(file_path, delimiter=delimiter, nrows=num_lines)
        print(df)
    except Exception as e:
        print(f"Error loading preview: {e}")

# preview_large_csv("Data/x_test.txt", delimiter='\t')

def check_nulls_in_data(x_file_path, y_file_path):
    def read_file_smart(path, label):
        try:
            df = pd.read_csv(path)
            print(f"{label}: Read with commas")
        except pd.errors.ParserError:
            try:
                df = pd.read_csv(path, delimiter='\t')
                print(f"{label}: Read with tabs")
            except Exception as e:
                print(f"Error reading the {label} file: {e}")
                return None
        except Exception as e:
            print(f"Error reading the {label} file: {e}")
            return None
        return df

    X = read_file_smart(x_file_path, "X")
    y = read_file_smart(y_file_path, "y")

    if X is None or y is None:
        return

    if X.isnull().values.any() or y.isnull().values.any():
        print("Null found")
    else:
        print("No nulls found")

check_nulls_in_data('Data/x_test.txt', 'Data/y_test.txt')
check_nulls_in_data('Data/x_train.txt', 'Data/y_train.txt')



X: Read with tabs
y: Read with commas
Null found
X: Read with tabs
y: Read with commas
Null found


In [ ]:
#Function to preprocess the data
def preprocessXandY(X, y):
    lang_codes = ['ita', 'fra', 'eng', 'ind', 'spa'] #lang codes of what we will use Italian, French, English, Indonesian, Spanish
    return None



In [17]:
#Understanding the data set
import pandas as pd
import numpy as np

def file_to_np_array(path, label):
    try:
        df = pd.read_csv(path, sep='<NonExistenceSeparator>', header=None, engine='python')
        print(f"{label}: Read each line into row")
    except Exception as e:
        print(f"Error reading the {label} file: {e}")
        return None
    return df.to_numpy()


def clean_np_data(X, y):
    stacked = np.hstack((y, X)) # Stack y and X side by side
    print(stacked.shape)
    clean_stacked = stacked[~np.any(pd.isna(stacked), axis=1), :] # Remove empty values
    print(clean_stacked.shape)
    lang_codes = ['ita', 'fra', 'eng', 'ind', 'spa']
    true_clean = clean_stacked[np.isin(clean_stacked[:,0], lang_codes),:] # Remove all rows that aren't our target languages
    print(true_clean.shape)
    return true_clean[:,1], true_clean[:,0] # Return cleaned as X and y split again

def clean_filter_and_stack(X1_file, y1_file, X2_file, y2_file):
    X1_clean, y1_clean = clean_np_data(file_to_np_array(X1_file, X1_file), 
                                       file_to_np_array(y1_file, y1_file))
    X2_clean, y2_clean = clean_np_data(file_to_np_array(X2_file, X2_file), 
                                       file_to_np_array(y2_file, y2_file))
    return np.hstack((X1_clean, X2_clean)), np.hstack((y1_clean, y2_clean))

X_all, y_all = clean_filter_and_stack("Data/x_train.txt", 
                                      "Data/y_train.txt", 
                                      "Data/x_test.txt", 
                                      "Data/y_test.txt")

print(X_all.shape, y_all.shape)
print(y_all[0])
    

Data/x_train.txt: Read each line into row
Data/y_train.txt: Read each line into row
(117500, 2)
(117000, 2)
(2500, 2)
Data/x_test.txt: Read each line into row
Data/y_test.txt: Read each line into row
(117500, 2)
(117000, 2)
(2500, 2)
(5000,) (5000,)
ind


In [4]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X_all, y_all, test_size=0.3, random_state=17)
print(X_tr.shape, y_tr.shape)
print(X_te.shape, y_te.shape)

(3500,) (3500,)
(1500,) (1500,)


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(strip_accents='unicode')
X_tr_vectors = vectorizer.fit_transform(X_tr)
X_te_vectors = vectorizer.transform(X_te)
print("Done vectorizing")

Done vectorizing


In [15]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_tr_vectors, y_tr)
print("Done training MNB")

Done training MNB


In [19]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_te_vectors)
accuracy = accuracy_score(y_te, y_pred)
print("Accuracy: ", accuracy)
print(y_pred[0:10])
print(X_te[0:10])

Accuracy:  0.9893333333333333
['fra' 'fra' 'ind' 'ind' 'spa' 'spa' 'ita' 'eng' 'ita' 'eng']
['Les magnolias ou magnoliers (Magnolia L., 1753) forment un genre de plantes à fleurs, de la famille des magnoliacées, qui comprend environ cent dix espèces, essentiellement des arbres et arbustes, des régions tempérées chaudes.'
 'Honoré Beaugrand, fils de Louis Beaugrand, dit Champagne, navigateur, et de Marie-Josephte (Joséphine) Marion, est l’aîné des fils d’une famille qui comptera six enfants (3 garçons et 3 filles).'
 'Sesudah itu, masih juga terdapat penyerangan-penyerangan, seperti ke Avignon (tahun 734), ke Lyon (tahun 743), dan pulau-pulau yang terdapat di Laut Tengah, Mallorca, Korsika, Sardegna, Kreta, Rhodos, Siprus dan sebagian dari Sisilia juga jatuh ke tangan Islam pada zaman Bani Umayyah. Gelombang kedua terbesar dari penyerbuan kaum Muslimin yang geraknya dimulai pada permulaan abad ke-8 M ini, telah menjangkau seluruh Spanyol dan melebar jauh menjangkau Perancis Tengah dan b